 # **Oriana Felszer**: 345143655  
 # **Aron Bensimhon**: 342762689
 
 https://github.com/orifelszer/TeslaHW

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.ad.co.il/car?sp261=69868"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
soup.find_all('div', class_='card-block')


[<div class="card-block" data-id="16186570" data-images="11" onclick="clickad(16186570)">
 <div class="card overflow-hidden">
 <picture class="card-img-top picbg">
 <source media="(max-width: 767px)" srcset="//img3.ad.co.il/CarImages/3935038-300_0.jpg"/>
 <source srcset="//img3.ad.co.il/CarImages/3935038-400_0.jpg"/>
 <img alt="" class="img-fluid" decoding="async" loading="lazy" onload="this.style.opacity = 1" src="//img3.ad.co.il/CarImages/3935038-400_0.jpg"/>
 </picture>
 <div class="card-body p-md-3">
 <a href="/ad/16186570"><h2 class="card-title mb-0 mb-sm-1">טסלה מודל 3</h2></a>
 <div class="card-text my-1 mb-1 mt-0 d-flex justify-content-between">
 <div>
                         2021
                     </div>
 <div>
                         5%<i class="fas fa-long-arrow-alt-down ps-1 pe-2"></i>
 <span class="text-decoration-line-through text-400"></span>
 </div>
 </div>
 <div class="d-flex justify-content-between">
 <div class="d-inline-flex">
 <div class="me-2 flex-fill card-i

In [3]:
manufacturer = soup.find('li',{'class':'breadcrumb-item active'}).find('span').get_text()
manufacturer

'טסלה'

In [4]:
url = "https://www.ad.co.il/car?keyword=" + manufacturer
response = requests.get(url)
results_page = BeautifulSoup(response.content, 'html.parser')


In [5]:
def fetch_tesla_cars(keywords):
    tesla_list = list()
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.ad.co.il/car?keyword=" + keywords
    response = requests.get(url)
    if not response.status_code == 200:
        print ("not 200")
        return None
    try:
        results_page = BeautifulSoup(response.content, 'html.parser')
        cars = results_page.find_all('div',{'class':'card-body p-md-3'})
        for car in cars:
            car_link = "https://www.ad.co.il" + car.find('a').get('href')
            car_name = car.find('a').get_text()
            
            if keywords in car_name:
                tesla_list.append((car_link)) 
        return tesla_list
    except:
        return None

In [6]:
list_of_links = fetch_tesla_cars(manufacturer)


In [7]:
def fetch_details(car_link):
    from datetime import datetime
    tesla_dict = {}
    counter = 0

    try:
        response = requests.get(car_link)
        if not response.status_code == 200:
            return "str"
        result_page = BeautifulSoup(response.content, 'html.parser')

        table = result_page.find('table', {'class': 'table table-sm mb-4'}).find('tbody')
        for tr in table.find_all("tr"):
            td = tr.find_all('td')

            key = td[0].get_text().strip()
            value = td[1].get_text().strip()
            if key == "שנה":
                value = int(value)
            elif key == "יד":
                value = int(value)
            elif key == "Pic_num":
                value = int(value)
            elif key == "נפח":
                value = int(''.join(filter(str.isdigit, value)))

            tesla_dict[key] = value

        picture = result_page.find_all('div', {'class': 'justify-content-center px-1'})
        tesla_dict['Pic_num'] = int(len(picture))

        model_and_price = result_page.find_all('h2', {'class': 'card-title'})
        tesla_dict['Price'] = model_and_price[1].get_text().strip().replace('₪', '')
        tesla_dict['model'] = model_and_price[0].get_text()

        Repub_date_and_Cre_date = result_page.find_all('div', {'class': 'px-3'})
        cre_date = Repub_date_and_Cre_date[0].get_text().split()
        Repub_date = Repub_date_and_Cre_date[1].get_text().split()
        tesla_dict['Cre_date'] = cre_date[2]
        tesla_dict['Repub_date'] = Repub_date[3]
        datetime.strptime(car_dict['Cre_date'], "%Y-%m-%d")
        datetime.strptime(car_dict['Repub_date'], "%Y-%m-%d")
        tesla_dict["manufacturer"] = "טסלה"

        description = result_page.find('p', {'class': 'text-word-break'}).get_text()

        return tesla_dict
    except:
        return tesla_dict


In [8]:
fetch_details('https://www.ad.co.il/ad/16186570')

{'שנה': 2021,
 'יד': 1,
 'ת. הילוכים': 'אוטומטית',
 'נפח': 6000,
 'סוג מנוע': 'חשמלי',
 'ק"מ': '48,000',
 'טסט עד': '06/2025',
 'צבע': 'שחור',
 'בעלות קודמת': 'פרטית',
 'בעלות נוכחית': 'פרטית',
 'אזור': 'ראשל"צ והסביבה',
 'עיר': 'באר יעקב',
 'Pic_num': 11,
 'Price': '190,000 ',
 'model': 'טסלה מודל 3',
 'Cre_date': '20/05/2024',
 'Repub_date': '01/06/2024'}

In [9]:
def fetch_all(keywords):
    results = list()
    all_teslas = fetch_tesla_cars(keywords)
    for car in all_teslas:
        t = fetch_details(car)
        results.append(t)
        
    return results

In [10]:
tesla = fetch_all(manufacturer)

In [11]:
tesla

[{'שנה': 2021,
  'יד': 1,
  'ת. הילוכים': 'אוטומטית',
  'נפח': 6000,
  'סוג מנוע': 'חשמלי',
  'ק"מ': '48,000',
  'טסט עד': '06/2025',
  'צבע': 'שחור',
  'בעלות קודמת': 'פרטית',
  'בעלות נוכחית': 'פרטית',
  'אזור': 'ראשל"צ והסביבה',
  'עיר': 'באר יעקב',
  'Pic_num': 11,
  'Price': '190,000 ',
  'model': 'טסלה מודל 3',
  'Cre_date': '20/05/2024',
  'Repub_date': '01/06/2024'},
 {'שנה': 2022,
  'יד': 1,
  'ת. הילוכים': 'אוטומטית',
  'נפח': 800,
  'סוג מנוע': 'חשמלי',
  'ק"מ': '33,000',
  'צבע': 'לבן',
  'בעלות קודמת': 'פרטית',
  'בעלות נוכחית': 'פרטית',
  'אזור': 'רעננה - כפר סבא',
  'עיר': 'כפר סבא',
  'Pic_num': 10,
  'Price': '145,000 ',
  'model': 'טסלה מודל 3',
  'Cre_date': '21/01/2024',
  'Repub_date': '22/05/2024'},
 {'שנה': 2018,
  'יד': 2,
  'ת. הילוכים': 'רובוטית',
  'נפח': 1000,
  'סוג מנוע': 'חשמלי',
  'צבע': 'שחור',
  'בעלות קודמת': 'פרטית',
  'בעלות נוכחית': 'פרטית',
  'אזור': 'רמת גן - גבעתיים',
  'עיר': 'בני ברק',
  'Pic_num': 5,
  'Price': '13,000 ',
  'model': 'טסלה מוד

In [12]:
import numpy as np
df = pd.DataFrame(tesla)
df = df.rename(columns={'שנה': 'Year', 'יד': 'Hand','ת. הילוכים': 'Gear', 'נפח': 'Engine_capacity','סוג מנוע': 'Engine_type', 'ק"מ': 'Km','טסט עד': 'Test', 'צבע': 'Color','עיר': 'City', 'אזור': 'Area','בעלות נוכחית':'Prev_ownership','בעלות קודמת':'Curr_ownership'})
df['manufacturer'] = 'Tesla'
df['Engine_capacity'] = df['Engine_capacity'].replace([np.inf, -np.inf], np.nan).fillna(0)
df['Engine_capacity'] = df['Engine_capacity'].astype(int)
df['Test']

0     06/2025
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6         NaN
7         NaN
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13        NaN
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
19        NaN
20        NaN
21        NaN
22        NaN
23        NaN
24        NaN
25        NaN
26    12/2024
27        NaN
28        NaN
Name: Test, dtype: object

In [13]:
df['Cre_date'] = pd.to_datetime(df['Cre_date'],dayfirst=True)
df['Test'] = pd.to_datetime(df['Test'],dayfirst=True)
df['Gear'] = pd.Categorical(df['Gear'])
df['Km'] = pd.to_numeric(df['Km'], errors='coerce').fillna(0).astype(int)
df['Engine_type'] = pd.Categorical(df['Engine_type'])
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'])
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'])
df['Area'] = df['Area'].astype(str)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0).astype(float)
df['Repub_date'] = pd.to_datetime(df['Repub_date'],dayfirst=True)
df['manufacturer'] = 'Tesla'
ordered_list = ['manufacturer','Year', 'Hand', 'Gear', 'Engine_capacity', 'Engine_type', 'Km', 'Test', 'Color',
     'Curr_ownership', 'Prev_ownership', 'Area', 'City', 'Pic_num', 'Price', 'model',
     'Cre_date', 'Repub_date']


df = df.reindex(columns=ordered_list)


In [14]:
df.to_csv('tesla.csv', index = False)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   manufacturer     29 non-null     object        
 1   Year             29 non-null     int64         
 2   Hand             29 non-null     int64         
 3   Gear             29 non-null     category      
 4   Engine_capacity  29 non-null     int64         
 5   Engine_type      29 non-null     category      
 6   Km               29 non-null     int64         
 7   Test             2 non-null      datetime64[ns]
 8   Color            18 non-null     object        
 9   Curr_ownership   18 non-null     category      
 10  Prev_ownership   18 non-null     category      
 11  Area             29 non-null     object        
 12  City             29 non-null     object        
 13  Pic_num          29 non-null     int64         
 14  Price            29 non-null     float64    

In [16]:
#test to num of days

In [17]:
def testings(car_link):
    try:
        response = requests.get(car_link)
        if response.status_code != 200:
            return "str"
        
        result_page = BeautifulSoup(response.content, 'html.parser')
        table = result_page.find('div', {'class': 'order-first order-sm-0'}).find('div')
        
        # Extract text from <p> elements with class 'text-word-break'
        text_list = []
        for p in table.find_all('p', class_='text-word-break'):
            text_list.append(p.get_text(strip=True))
        return text_list[0].replace('\n', '').replace('\r', '')
    except Exception as e:
        return str(e)

In [18]:
testings('https://www.ad.co.il/ad/16186570')

'מודל 3 פרפורמנס מטוס קרב אמיתיצבע מקורי שחור מבריק פנים לבןציפוי ppf מט מהיום הראשוןppf מבריק פנסים וגגהשחרה בפנסים אחוריים,חלונות,סמלים וגג מסך מתכוונן לנוחות מירביתספוילר מורחב מראות Mמחיר 190000 סופי לסגור לרציניים'

In [19]:
def fetch_all_links(list_of_links):
    list_link = []
    for i in list_of_links:
        list_link.append(testings(i))
    return list_link

In [20]:
fetch_all_links(list_of_links)

['מודל 3 פרפורמנס מטוס קרב אמיתיצבע מקורי שחור מבריק פנים לבןציפוי ppf מט מהיום הראשוןppf מבריק פנסים וגגהשחרה בפנסים אחוריים,חלונות,סמלים וגג מסך מתכוונן לנוחות מירביתספוילר מורחב מראות Mמחיר 190000 סופי לסגור לרציניים',
 'ג\'ילי גיאומטרי 460Pro עד 485ק"מ בין טעינות. אם לא זמין נא לסמסדגם מפואר ואמין, 480ק"מ בטעינה אחת.קירוי נפתח חשמלי.מושבי-בד רכים מחוממים וחשמליים 6 זכרונות.חדש! שמור ונקי, ללא עישון. תאורת אוירה.צבע בלמים ירוק בהיר. הכל חשמלי ומואר, מקורי וכלול.מסך 12.5" ווייז, נטפליקס, דיסני וספוטיפיי בנסיעה.אינטרנט אלחוטי מובנה ומצלמות 360 מקליטות בנסיעה.רמקולים וסאב מובנים BOZE.גלגל רזרבי מתחת לכלים בבגאז\' יש גם כבל טעינה ציבורית.',
 'בימבה חשמלית ⚡️⚡️⚡️הדבר הבא!!! מנוע -1200בקר -3 אלף גרין ביק סוללה - 72v35a חזקה מאוד נוסע איזה 50 55 קמ״ש מהיר מאוד מחזיקה 40 קילומטר משהו כזה מפלצת ללא רשיון ✌️',
 'טסלה מודל 3 לונג ריינג 2021 לבן פנינה הנעה 4x4498 כוח סוסהרכב במצב שמור מאוד מבחוץ ומבפנים.לבוא ולקחת.',
 'מציאה אמיתית! רכב חדש חדש במצב כמו להוציא מהחברה רק ב25 אלף שקל פחות. תוספת 

In [21]:
a = fetch_all_links(list_of_links)
df['Description'] = a
df

,manufacturer,Year,Hand,Gear,Engine_capacity,Engine_type,Km,Test,Color,Curr_ownership,Prev_ownership,Area,City,Pic_num,Price,model,Cre_date,Repub_date,Description
0,Tesla,2021,1,אוטומטית,6000,חשמלי,0,2025-06-01,שחור,פרטית,פרטית,"ראשל""צ והסביבה",באר יעקב,11,0.0,טסלה מודל 3,2024-05-20,2024-06-01,מודל 3 פרפורמנס מטוס קרב אמיתיצבע מקורי שחור מ...
1,Tesla,2022,1,אוטומטית,800,חשמלי,0,NaT,לבן,פרטית,פרטית,רעננה - כפר סבא,כפר סבא,10,0.0,טסלה מודל 3,2024-01-21,2024-05-22,"ג'ילי גיאומטרי 460Pro עד 485ק""מ בין טעינות. אם..."
2,Tesla,2018,2,רובוטית,1000,חשמלי,0,NaT,שחור,פרטית,פרטית,רמת גן - גבעתיים,בני ברק,5,0.0,טסלה מודל S,2024-03-22,2024-03-22,בימבה חשמלית ⚡️⚡️⚡️הדבר הבא!!! מנוע -1200בקר -...
3,Tesla,2021,2,אוטומטית,4900,חשמלי,0,NaT,לבן פנינה,פרטית,פרטית,נס ציונה - רחובות,רחובות,4,0.0,טסלה מודל 3,2023-09-02,2023-09-02,טסלה מודל 3 לונג ריינג 2021 לבן פנינה הנעה 4x4...
4,Tesla,2023,2,אוטומטית,6000,חשמלי,0,NaT,לבן פנינה,פרטית,פרטית,ירושלים והסביבה,ירושלים,1,0.0,טסלה מודל 3,2023-08-16,2023-08-16,מציאה אמיתית! רכב חדש חדש במצב כמו להוציא מהחב...
5,Tesla,2023,1,אוטומטית,3000,חשמלי,0,NaT,לבן שנהב,חברה,חברה,חדרה וישובי עמק חפר,חדרה,8,0.0,טסלה מודל X,2023-06-16,2023-06-16,חדש אנרגיה רכב עיר2023latest neta s עראבה חשמל...
6,Tesla,2021,1,אוטומטית,5000,חשמלי,0,NaT,לבן פנינה,פרטית,פרטית,עמק יזרעאל,קרית טבעון,2,0.0,טסלה מודל 3,2023-04-10,2023-04-14,"טסלה ייצור ארה""ב כולל וו גרירה. שמורה מאד מאד"
7,Tesla,2022,2,אוטומטית,6000,חשמלי,0,NaT,כחול,פרטית,פרטית,רמת גן - גבעתיים,רמת גן,3,0.0,טסלה מודל 3,2023-02-08,2023-02-14,"למכירה:מודל 3 LR לונג ריינג׳ , 498 כ״ס 4x4, טו..."
8,Tesla,2022,1,אוטומטית,6000,חשמלי,0,NaT,לבן פנינה,פרטית,פרטית,ירושלים והסביבה,שורש,4,0.0,טסלה מודל S,2023-01-18,2023-01-18,"על הכביש מסוף דצמבר 22, נדירה, בחניה פרטית"
9,Tesla,2021,1,אוטומטית,0,חשמלי,0,NaT,NaN,NaN,NaN,באר שבע והסביבה,באר שבע,1,0.0,טסלה מודל 3,2022-10-17,2022-10-17,טסלה מודל 3 פרפרומנסהדגם הכי חזק יפה ומפואר513...


In [28]:

# Get today's date
today = pd.to_datetime('today')

# Calculate the difference in days
df['days_until_today'] = (today - df['Test']).dt.days
df['days_until_today'] = df['days_until_today']*(-1)
df['days_until_today'] = df['days_until_today'].replace([np.inf, -np.inf], np.nan).fillna(0)
df['days_until_today'] = df['days_until_today'].astype(int)
df = df.drop(columns=['Test'])
df['Test'] = df['days_until_today']
df = df.drop(columns=['days_until_today'])


In [23]:
#df = df.drop(columns=['days_until_today'])

In [25]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   manufacturer     29 non-null     object        
 1   Year             29 non-null     int64         
 2   Hand             29 non-null     int64         
 3   Gear             29 non-null     category      
 4   Engine_capacity  29 non-null     int64         
 5   Engine_type      29 non-null     category      
 6   Km               29 non-null     int64         
 7   Test             2 non-null      datetime64[ns]
 8   Color            18 non-null     object        
 9   Curr_ownership   18 non-null     category      
 10  Prev_ownership   18 non-null     category      
 11  Area             29 non-null     object        
 12  City             29 non-null     object        
 13  Pic_num          29 non-null     int64         
 14  Price            29 non-null     float64    

In [30]:
df.to_csv('tesla.csv', index = False)